In [2]:
from encoded_client.encoded import ENCODED
import pdb
import pandas as pd
import glob
import pysam
import numpy as np
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

In [3]:
config_file = 'snakemake/config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

In [15]:
ab = '../'+expand(config['data']['ab'], species='human')[0]
filt_ab = expand(config['data']['filt_ab'], species='human')[0]
gtf = expand(config['data']['cerb_gtf'], species='human')[0]
read_annot = '../'+expand(config['data']['read_annot'], species='human')[0]
t_metadata = '../'+expand(config['ref']['cerberus']['t_info'], species='human')[0]
lib_meta = '../'+expand(config['data']['meta'], species='human')[0]
swan_file = '../'+expand(config['data']['sg'], species='human')[0]
cerberus_h5 = expand(config['data']['cerb_annot'], species='human')[0]
cerb_t_metadata = '../'+expand(config['data']['t_info'], species='human')[0]
major_isos = '../'+expand(config['data']['major_isos'], species='human', obs_col='sample')[0]
pp_summary = '../'+expand(config['data']['p_pred']['summary'], species='human')[0]
ref_t_metadata = '../'+expand(config['ref']['t_info'], species='human')[0]
ref_g_metadata = '../'+expand(config['ref']['g_info'], species='human')[0]

ver = 'v40_cerberus'
min_tpm = 1
gene_subset = 'protein_coding'
obs_col = 'sample'
go_gene_subset = 'protein_coding'

m_lib_meta = '../'+expand(config['data']['meta'], species='mouse')[0]

## 231009 adding pertinent information to the filtered biggenepred or w/e

In [119]:
sample = 'huvec'
bgp = expand(config['data']['tracks']['sample']['bgp_sort_filt'], species='human', sample='huvec')[0]

In [120]:
# get transcript name of each thing / novelty / novelty of each of three pieces?
df = pd.read_csv(bgp, sep='\t', header=None)

In [121]:
df.rename({3:'transcript_id'}, axis=1, inplace=True)

In [122]:
# ca = cerberus.read(cerberus_h5)

In [123]:
t_map = ca.t_map.copy(deep=True)

tss = ca.tss.copy(deep=True)
tss.rename({'Name': 'tss_id', 
            'novelty': 'tss_novelty'}, 
             axis=1, inplace=True)
tss = tss[['tss_id', 'tss_novelty']]

tes = ca.tes.copy(deep=True)
tes.rename({'Name': 'tes_id',
            'novelty': 'tes_novelty'}, 
           axis=1, inplace=True)
tes = tes[['tes_id', 'tes_novelty']]

ic = ca.ic.copy(deep=True)
ic.rename({'Name': 'ic_id', 
           'novelty': 'ic_novelty'}, 
          axis=1, inplace=True)
ic = ic[['ic_id', 'ic_novelty']]

t_map = t_map.merge(tss, how='left', on='tss_id')
t_map = t_map.merge(tes, how='left', on='tes_id')
t_map = t_map.merge(ic, how='left', on='ic_id')

t_map = t_map[['transcript_id', 'gene_name', 'transcript_name',
               'tss_id', 'ic_id', 'tes_id', 
               'tss_novelty', 'ic_novelty', 'tes_novelty']]
t_map = t_map.drop_duplicates()

In [124]:
df = df.merge(t_map,
              how='left',
              on='transcript_id')

In [125]:
df.head()

,0,1,2,transcript_id,4,5,6,7,8,9,...,18,19,gene_name,transcript_name,tss_id,ic_id,tes_id,tss_novelty,ic_novelty,tes_novelty
0,chr1,14353,29408,"ENSG00000227232[2,5,1]",0,-,29408,29408,0,11,...,ENSG00000227232.5,NaN,WASH7P,"WASH7P[2,5,1]",ENSG00000227232_2,ENSG00000227232_5,ENSG00000227232_1,Novel,NNC,Known
1,chr1,14353,29620,"ENSG00000227232[1,2,1]",0,-,29620,29620,0,9,...,ENSG00000227232.5,NaN,WASH7P,"WASH7P[1,2,1]",ENSG00000227232_1,ENSG00000227232_2,ENSG00000227232_1,Known,NNC,Known
2,chr1,184823,199925,"ENSG00000279457[2,6,2]",0,-,199925,199925,0,11,...,ENSG00000279457.4,NaN,WASH9P,"WASH9P[2,6,2]",ENSG00000279457_2,ENSG00000279457_6,ENSG00000279457_2,Novel,NNC,Novel
3,chr1,184823,199925,"ENSG00000279457[2,9,2]",0,-,199925,199925,0,11,...,ENSG00000279457.4,NaN,WASH9P,"WASH9P[2,9,2]",ENSG00000279457_2,ENSG00000279457_9,ENSG00000279457_2,Novel,NNC,Novel
4,chr1,185166,195461,"ENSG00000279457[1,1,1]",0,-,195461,195461,0,10,...,ENSG00000279457.4,NaN,WASH9P,"WASH9P[1,1,1]",ENSG00000279457_1,ENSG00000279457_1,ENSG00000279457_1,Known,Known,Known


In [102]:
# add gene biotype from gencode
t_df, _, _ = get_gtf_info(how='iso', ver=ver, add_stable_gid=True)

In [103]:
df['gid_stable'] = cerberus.get_stable_gid(df, 18)

In [104]:
t_df.rename({'tid':'transcript_id'}, axis=1, inplace=True)

In [105]:
df = df.merge(t_df[['MANE_Select', 'MANE_Plus_Clinical', 't_len', 'transcript_id']],
              how='left',
              on='transcript_id')

In [106]:
gene_df, _, _ = get_gtf_info(how='gene', ver=ver, add_stable_gid=True)

In [108]:
df

,0,1,2,transcript_id,4,5,6,7,8,9,...,17,18,19,gene_name,transcript_name,gid_stable_x,MANE_Select,MANE_Plus_Clinical,gid_stable_y,t_len
0,chr1,14353,29408,"ENSG00000227232[2,5,1]",0,-,29408,29408,0,11,...,"ENSG00000227232[2,5,1]",ENSG00000227232.5,NaN,WASH7P,"WASH7P[2,5,1]",ENSG00000227232,NaN,NaN,NaN,NaN
1,chr1,14353,29620,"ENSG00000227232[1,2,1]",0,-,29620,29620,0,9,...,"ENSG00000227232[1,2,1]",ENSG00000227232.5,NaN,WASH7P,"WASH7P[1,2,1]",ENSG00000227232,NaN,NaN,NaN,NaN
2,chr1,184823,199925,"ENSG00000279457[2,6,2]",0,-,199925,199925,0,11,...,"ENSG00000279457[2,6,2]",ENSG00000279457.4,NaN,WASH9P,"WASH9P[2,6,2]",ENSG00000279457,NaN,NaN,NaN,NaN
3,chr1,184823,199925,"ENSG00000279457[2,9,2]",0,-,199925,199925,0,11,...,"ENSG00000279457[2,9,2]",ENSG00000279457.4,NaN,WASH9P,"WASH9P[2,9,2]",ENSG00000279457,NaN,NaN,NaN,NaN
4,chr1,185166,195461,"ENSG00000279457[1,1,1]",0,-,195461,195461,0,10,...,"ENSG00000279457[1,1,1]",ENSG00000279457.4,NaN,WASH9P,"WASH9P[1,1,1]",ENSG00000279457,False,False,ENSG00000279457,1507.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48747,chrY,19705366,19744811,"ENSG00000012817[1,1,2]",0,-,19744811,19744811,0,27,...,"ENSG00000012817[1,1,2]",ENSG00000012817.15,NaN,KDM5D,"KDM5D[1,1,2]",ENSG00000012817,NaN,NaN,NaN,NaN
48748,chrY,19705366,19744989,"ENSG00000012817[2,1,2]",0,-,19744989,19744989,0,27,...,"ENSG00000012817[2,1,2]",ENSG00000012817.15,NaN,KDM5D,"KDM5D[2,1,2]",ENSG00000012817,NaN,NaN,NaN,NaN
48749,chrY,20575725,20592680,"ENSG00000198692[1,1,2]",0,+,20592680,20592680,0,7,...,"ENSG00000198692[1,1,2]",ENSG00000198692.9,NaN,EIF1AY,"EIF1AY[1,1,2]",ENSG00000198692,NaN,NaN,NaN,NaN
48750,chrY,20575725,20592680,"ENSG00000198692[1,2,2]",0,+,20592680,20592680,0,6,...,"ENSG00000198692[1,2,2]",ENSG00000198692.9,NaN,EIF1AY,"EIF1AY[1,2,2]",ENSG00000198692,False,False,ENSG00000198692,870.0


In [107]:
df = df.merge(gene_df[['biotype_category', 'biotype', 'gid_stable']],
              how='left',
              on='gid_stable')

KeyError: 'gid_stable'

In [75]:
df

,0,1,2,transcript_id,4,5,6,7,8,9,...,16,17,18,19,gene_name,transcript_name,gid_stable,MANE_Select,MANE_Plus_Clinical,t_len
0,chr1,14353,29408,"ENSG00000227232[2,5,1]",0,-,29408,29408,0,11,...,NaN,"ENSG00000227232[2,5,1]",ENSG00000227232.5,NaN,WASH7P,"WASH7P[2,5,1]",ENSG00000227232,False,False,1462.0
1,chr1,14353,29620,"ENSG00000227232[1,2,1]",0,-,29620,29620,0,9,...,NaN,"ENSG00000227232[1,2,1]",ENSG00000227232.5,NaN,WASH7P,"WASH7P[1,2,1]",ENSG00000227232,False,False,1462.0
2,chr1,184823,199925,"ENSG00000279457[2,6,2]",0,-,199925,199925,0,11,...,NaN,"ENSG00000279457[2,6,2]",ENSG00000279457.4,NaN,WASH9P,"WASH9P[2,6,2]",ENSG00000279457,False,False,1507.0
3,chr1,184823,199925,"ENSG00000279457[2,9,2]",0,-,199925,199925,0,11,...,NaN,"ENSG00000279457[2,9,2]",ENSG00000279457.4,NaN,WASH9P,"WASH9P[2,9,2]",ENSG00000279457,False,False,1507.0
4,chr1,185166,195461,"ENSG00000279457[1,1,1]",0,-,195461,195461,0,10,...,NaN,"ENSG00000279457[1,1,1]",ENSG00000279457.4,NaN,WASH9P,"WASH9P[1,1,1]",ENSG00000279457,False,False,1507.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639443,chrY,20575725,20593204,"ENSG00000198692[1,1,1]",0,+,20593204,20593204,0,7,...,NaN,"ENSG00000198692[1,1,1]",ENSG00000198692.9,NaN,EIF1AY,"EIF1AY[1,1,1]",ENSG00000198692,True,False,1446.0
639444,chrY,20575725,20593204,"ENSG00000198692[1,1,1]",0,+,20593204,20593204,0,7,...,NaN,"ENSG00000198692[1,1,1]",ENSG00000198692.9,NaN,EIF1AY,"EIF1AY[1,1,1]",ENSG00000198692,False,False,870.0
639445,chrY,20575725,20593204,"ENSG00000198692[1,1,1]",0,+,20593204,20593204,0,7,...,NaN,"ENSG00000198692[1,1,1]",ENSG00000198692.9,NaN,EIF1AY,"EIF1AY[1,1,1]",ENSG00000198692,False,False,962.0
639446,chrY,20575725,20593204,"ENSG00000198692[1,1,1]",0,+,20593204,20593204,0,7,...,NaN,"ENSG00000198692[1,1,1]",ENSG00000198692.9,NaN,EIF1AY,"EIF1AY[1,1,1]",ENSG00000198692,False,False,3250.0


In [76]:
gene_df.head()

,gid,gname,length,biotype,biotype_category,tf,MANE_Select,MANE_Plus_Clinical,gid_stable
0,ENSG00000000460.17,C1orf112,192174,protein_coding,protein_coding,False,True,False,ENSG00000000460
1,ENSG00000000971.17,CFH,100823,protein_coding,protein_coding,False,True,False,ENSG00000000971
2,ENSG00000001461.17,NIPAL3,57275,protein_coding,protein_coding,False,True,False,ENSG00000001461
3,ENSG00000004487.18,KDM1A,64347,protein_coding,protein_coding,False,True,False,ENSG00000004487
4,ENSG00000007933.13,FMO3,27019,protein_coding,protein_coding,False,True,False,ENSG00000007933


In [ ]:
# also add predicted CDS start / ends

In [ ]:
# add tpm

## 231004 making a GTF for each sample

In [8]:
df = pd.read_csv(filt_ab, sep='\t')
df = get_det_table(df,
                   groupby=obs_col,
                   how='iso',
                   min_tpm=min_tpm)

Calculating iso TPM values


/Users/fairliereese/Documents/programming/mortazavi_lab/data/paper_rnawg/scripts/utils.py:2176: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tpm_col] = (df[d]*1000000)/df[total_col]
/Users/fairliereese/Documents/programming/mortazavi_lab/data/paper_rnawg/scripts/utils.py:2175: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[total_col] = df[d].sum()


Enforcing minimum TPM
Total # isos detected: 231136
# isos >= 1 tpm: 210239
Number of isos reported: 210239
Found 50 total samples


In [10]:
df = df.transpose()

In [13]:
print(len(df.index))
for c in df.columns:
    tids = df.loc[df[c]==True].index.tolist()
    # print(len(tids))
    # print(tids[:5])
    # break
    

210239
56341
['ENSG00000279457[1,1,1]', 'ENSG00000279457[2,4,2]', 'ENSG00000279457[2,6,2]', 'ENSG00000279457[2,9,2]', 'ENSG00000225880[1,1,1]']


In [15]:
gtf_df = pr.read_gtf(gtf, rename_attr=True).as_df()

Found attributes with reserved names: ['source'].
Renaming attributes with suffix '_attr'


In [17]:
gids = gtf_df.loc[gtf_df.transcript_id.isin(tids), 'gene_id'].tolist()

In [19]:
# gids[:5]

In [20]:
gtf_df = gtf_df.loc[gtf_df.transcript_id.isin(tids)]

In [21]:
gtf_df = pr.PyRanges(gtf_df)

In [22]:
gtf_df.to_gtf('test_sample_gtf.gtf')

## snakemake debug

In [5]:
species = ['human', 'mouse']
meta_df = get_meta_df(config, species)

In [7]:
meta_df.loc[meta_df.dataset.str.contains('adrenal_36d_m')]

,ENCODE_experiment_id,dataset,sample,sample_display,general_tissue_cell_type,fig1_tissue_label,health_status,tissue_or_cell_line,sample_color_hex_code,matching_mouse_samples,...,document_labels,platform,RIN,spikeins,species,age,adult,sex,genotype,matching_human_samples
19,ENCSR242UEB,adrenal_36d_m_2,adrenal_36d,Adrenal gland PND36,adrenal gland,adrenal gland,NaN,tissue,#c87961,NaN,...,"'PacBio libraries v3 (October, 2020) Protocol ...",Pacific Biosciences Sequel II,6.4,True,mouse,36d,True,m,b6/cast,['adrenal gland']
20,ENCSR242UEB,adrenal_36d_m_1,adrenal_36d,Adrenal gland PND36,adrenal gland,adrenal gland,NaN,tissue,#c87961,NaN,...,"'PacBio libraries v3 (October, 2020) Protocol ...",Pacific Biosciences Sequel II,6.4,True,mouse,36d,True,m,b6/cast,['adrenal gland']


In [ ]:
# 230722 test get_lr_ids

In [13]:
def get_lr_exp_meta():
    def get_genome_bam(experiment_id):
        experiment = server.get_json(experiment_id)

        metadata = []
        default_analysis = experiment["default_analysis"]
        for f in experiment["files"]:
            analyses = [x["@id"] for x in f["analyses"]]
            # this check only applies to processed data not reads (default_analysis in analyses and)
            if f["output_type"] in ("reads", "unfiltered alignments"):
                if f["output_type"] == "unfiltered alignments" and not default_analysis in analyses:
                    # skip alignments for older analyses.
                    continue
                reps = f["technical_replicates"]
                assert len(reps) == 1
                reps = reps[0]

                extension = {
                    "fastq": ".fastq.gz",
                    "bam": ".bam",
                }[f["file_format"]]
    
                metadata.append({
                    "experiment": experiment["accession"],
                    "description": experiment["description"],
                    "simple_biosample_summary": f["simple_biosample_summary"],
                    "file": f["accession"],
                    "output_type": f["output_type"],
                    "file_size": f["file_size"],
                    "bio_rep": f["biological_replicates"][0],
                    "tech_rep": f["technical_replicates"][0]
                })

        return metadata

    server = ENCODED("www.encodeproject.org") 
    cart_url = "https://www.encodeproject.org/carts/829d339c-913c-4773-8001-80130796a367/"    
    
    cart = server.get_json(cart_url)

    metadata = []
    for experiment_id in cart["elements"]:
        metadata.extend(get_genome_bam(experiment_id))
        
    metadata = pd.DataFrame(metadata)
    metadata['name'] = metadata['experiment']+'_'+metadata['tech_rep']
    return metadata    

In [5]:
def get_lr_fastq_ids(file_format='bam'):
    metadata = get_lr_exp_meta()
    if file_format == 'bam':
        metadata = metadata.loc[metadata.output_type=='unfiltered alignments']
    elif file_format == 'fastq':
        metadata = metadata.loc[metadata.output_type=='reads']
    
    return metadata.file.tolist()

In [6]:
ids = get_lr_fastq_ids(file_format='bam')

In [7]:
len(ids)

138

In [25]:
ids = get_lr_fastq_ids(file_format='fastq')

In [26]:
len(ids)

138

In [33]:
def score_aligned_reads(filename):
    with pysam.AlignmentFile(filename, "rb") as inbam:
        aligned_reads = 0
        aligned_query_len = []
        aligned_positions = []

        for read in inbam:
            if not read.is_unmapped:
                aligned_reads += 1
                aligned_query_len.append(read.query_length)
                aligned_positions.append(len(read.positions))

        return {
            "aligned_reads": len(aligned_query_len),
            "query_len_median": np.median(aligned_query_len),
            "query_len": aligned_query_len,
            "positions_median": np.median(aligned_positions),
            "positions_len": aligned_positions,
        }
def score_fastq_reads(filename):
    read_len = []
    with xopen(filename, "rt") as instream:
        for record in SeqIO.parse(instream, "fastq"):
            read_len.append(len(record))
    
    return {
        "raw_reads": len(read_len),
        "read_len_median": np.median(read_len),
        "read_len": read_len,
    }

In [34]:
metadata = get_lr_exp_meta()
metadata.head()

,experiment,description,simple_biosample_summary,file,output_type,file_size,bio_rep,tech_rep,name
0,ENCSR293MOX,W62 right lobe of liver,female child (16 years),ENCFF318SKH,reads,255510907,1,1_1,ENCSR293MOX_1_1
1,ENCSR293MOX,W62 right lobe of liver,female child (16 years),ENCFF430RUG,unfiltered alignments,362638099,1,1_1,ENCSR293MOX_1_1
2,ENCSR081NRO,Adrenal Gland 1-2 ENTEX50,male adult (37 years),ENCFF211SQY,reads,387217252,1,1_1,ENCSR081NRO_1_1
3,ENCSR081NRO,Adrenal Gland 1-2 ENTEX50,male adult (37 years),ENCFF967OHL,unfiltered alignments,521377683,1,1_1,ENCSR081NRO_1_1
4,ENCSR257JBF,excitatory cortical neurons,male adult (53 years),ENCFF919JFJ,reads,345607718,1,1_1,ENCSR257JBF_1_1


In [35]:
counts = {}
d = 'data/human/bam/*bam'
for f in glob.glob(d):
    encid = f.rsplit('/', maxsplit=1)[1].split('.')[0]
    temp = metadata.loc[metadata.file == encid].iloc[0]
    name = temp.name
    if temp.output_type == 'unfiltered alignments':
        result = score_aligned_reads(f)
    elif temp.output_type == 'reads':
        result = score_fastq_reads(f)
    counts.setdefault(name, {}).update(result)

OSError: no BGZF EOF marker; file may be truncated

In [ ]:
array_attributes = ['read_len', 'query_len', 'positions_len']
lengths = {}

medians = {}
for name in counts:
    for k in counts[name]:
        if k in array_attributes:
            lengths[(k, name)] = counts[name][k]
        else:
            medians.setdefault(k, {})[name] = counts[name][k]

In [ ]:
def get_lr_read_lens(bams, fastqs, out):
    
    def score_aligned_reads(filename):
        with pysam.AlignmentFile(filename, "rb") as inbam:
            aligned_reads = 0
            aligned_query_len = []
            aligned_positions = []

            for read in inbam:
                if not read.is_unmapped:
                    aligned_reads += 1
                    aligned_query_len.append(read.query_length)
                    aligned_positions.append(len(read.positions))

            return {
                "aligned_reads": len(aligned_query_len),
                "query_len_median": np.median(aligned_query_len),
                "query_len": aligned_query_len,
                "positions_median": np.median(aligned_positions),
                "positions_len": aligned_positions,
            }
    def score_fastq_reads(filename):
        read_len = []
        with xopen(filename, "rt") as instream:
            for record in SeqIO.parse(instream, "fastq"):
                read_len.append(len(record))

        return {
            "raw_reads": len(read_len),
            "read_len_median": np.median(read_len),
            "read_len": read_len,
        }
    
    counts = {}
    for f in bams+fastqs:
        encid = f.rsplit('/', maxsplit=1)[1].split('.')[0]
        temp = metadata.loc[metadata.file == encid].iloc[0]
        name = temp.name
        if temp.output_type == 'unfiltered alignments':
            result = score_aligned_reads(f)
        elif temp.output_type == 'reads':
            result = score_fastq_reads(f)
        counts.setdefault(name, {}).update(result)
        
    array_attributes = ['read_len', 'query_len', 'positions_len']
    lengths = {}

    medians = {}
    for name in counts:
        for k in counts[name]:
            if k in array_attributes:
                lengths[(k, name)] = counts[name][k]
            else:
                medians.setdefault(k, {})[name] = counts[name][k]
                
    various_counts = pd.DataFrame(medians)
    various_counts.to_csv(out, sep='\t')
